In [ ]:
import os
import torch
import csv
from shutil import move
from torchvision.transforms import v2
from torch.utils.data import DataLoader
from timeit import default_timer as timer
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/YOLO')
from CircleYoloModule.utilities import yolo_training, map_utilities, dataset_utilities
from CircleYoloModule.circleyolov1_1_structure import Yolo
from CircleYoloModule.yolo_loss_circle import YoloLossCircleMultiBBox as Loss_fun

# HIPERPARAMETERS
BATCH_SIZE = 64
H, W = 448, 448  # Image transform size for the model
LEARNING_RATE = 1e-5
WEIGHT_DECAY = 1e-4
EPOCHS = 250
NUM_CLASS = 1
STRIDE = 7
BBOXES = 1
LAMBDA_NOOBJ = 0.5
LAMBDA_COORD = 5
IMG_CH_NUM = 3  # Number of image channels
PATH_TO_SAVE_MODEL = '/content/drive/MyDrive/!MojeYolov1/Modele_magisterka/model_yoloV1-circle_1BBox_noobj-0,5_coord-5.pth'
PATH_TO_SAVE_CHECKPOINT = '/content/drive/MyDrive/!MojeYolov1/checkpoint_yoloV1-circle_1BBox_noobj-0,5_coord-5.pth'
DST_PATH_END_CSV = '/content/drive/MyDrive/!MojeYolov1/Modele_magisterka/yolo1-1_epoch_loss_ap_1BBox_noobj-0,5_coord-5.csv'
START_EPOCH = 1  # epoch % 5 == 1
LOAD_MODEL = False
WEIGHTS_PATH = PATH_TO_SAVE_MODEL
# Paths to archives
ARH_PATH_0 = '/content/drive/MyDrive/!MojeYolov1/archiwumJabłkaParts.zip'  # Artificial Light Dataset
ARH_PATH_1 = '/content/drive/MyDrive/!MojeYolov1/archiwumMinnerApple.zip'
ARH_PATH_2 = '/content/drive/MyDrive/!MojeYolov1/archiwumACFR_multifruit-2016(apples_only).zip'

def main():
    # Setting seed for reproducibility
    torch.manual_seed(42)

    # Check device
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # Augmentation and normalization for images
    transforms = v2.Compose([
        v2.Resize(size=(H, W), antialias=True, interpolation=3),  # 3: InterpolationMode.BICUBIC
        v2.ColorJitter(brightness=(0.75, 1.5), contrast=(0.5, 1.5), saturation=(0.5, 1.5), hue=(-0.25, 0.25)),
        v2.ToDtype(torch.float32, scale=True),  # Normalize expects float input
    ])

    # Unpacking datasets
    dataset_train, dataset_val, dataset_test = dataset_utilities.unpackDatasets(
        '/content',  # Destiny dir
        transforms,  # Transforms
        'V1',  # Define its for YOLO_v1
        None,  # No anchor boxes for YOLO_v1
        (0.9, 0.05, 0.05),  # Split proportion to datasets: train, val, test
        ARH_PATH_0, ARH_PATH_1, ARH_PATH_2  # Paths to archives
    )

    # Loading datasets to dataloaders
    NUM_WORKERS = os.cpu_count()

    train_dataloader = DataLoader(dataset=dataset_train, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=True)
    val_dataloader = DataLoader(dataset=dataset_val, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=False)
    test_dataloader = DataLoader(dataset=dataset_test, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=False)

    # Setting model structure, YOLOv1
    model = Yolo(input_shape=IMG_CH_NUM, num_class=NUM_CLASS, segmentation_stride=STRIDE, num_bbox=BBOXES).to(device)
    model = torch.compile(model)

    # Setting model checkpoints for highest mAP value
    bestCheckpoint = yolo_training.ModelCheckpoint(filepath=PATH_TO_SAVE_MODEL, mode='max', monitor='AP')

    # Setting mixed policy
    use_amp = False

    # Setting loss
    loss_fn = Loss_fun(C=NUM_CLASS, S=STRIDE, B=BBOXES, lambda_noobj=LAMBDA_NOOBJ, lambda_coord=LAMBDA_COORD)

    # Setting optimizer Adam
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

    # Setting timer start to measure train time
    train_time_start_model = timer()

    # Function aliasing for training
    get_bboxes, mAP = map_utilities.get_bboxes, map_utilities.mean_average_precision
    print_time, train_step = yolo_training.print_train_time, yolo_training.train_step

    # Load model checkpoint
    global START_EPOCH
    if LOAD_MODEL:
        checkpoint = torch.load(PATH_TO_SAVE_CHECKPOINT, map_location=device)
        bestCheckpoint.set_mAP(checkpoint['best_mAP'])
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        START_EPOCH = checkpoint['epoch'] + 1

    # Training loop
    AP = 0
    for epoch in range(START_EPOCH, EPOCHS):
        print(f'Epoch: {epoch:-<70}')

        loss = train_step(model=model, data_loader=train_dataloader, yolo_loss_fn=loss_fn, scaler=None, optimizer=optimizer, device=device)

        if epoch % 5 == 1:
            pred_box, true_box = get_bboxes(model=model, loader=val_dataloader, use_amp=use_amp, device=device, IoU_threshold=0.5, threshold=0.5, S=7, C=1)
            # mAP @50
            AP = mAP(pred_boxes=pred_box, true_boxes=true_box, threshold_mAP=0.5, step_threshold=1, stop_threshold_mAP=0.95, C=NUM_CLASS, epsilon=1E-6)

            print(f"Average Precision @50. For val dataset is equal: {(AP * 100):.2f}%")
            bestCheckpoint(AP, model)
            if float(AP) >= 0.95:
                print('Early stopping. Average Precision @50 over 95')
                break

        train_time_end_model = timer()
        total_train_time_model = print_time(start=train_time_start_model, end=train_time_end_model, device=device)

        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_time': total_train_time_model,
            'best_mAP': bestCheckpoint.get_mAP()
        }

        # Save checkpoint
        torch.save(checkpoint, PATH_TO_SAVE_CHECKPOINT)
        print(f"Loss: {loss:.2f}")

        # Write result to CSV
        with open(DST_PATH_END_CSV, mode='a') as loss_file:
            csv_writer = csv.writer(loss_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            if epoch == 1:
                csv_writer.writerow(['Epoch', 'Loss', 'mAP', 'train_time[s]'])
            csv_writer.writerow([f'{epoch}', f'{loss}', f'{AP}', f'{total_train_time_model}'])

if __name__ == '__main__':
    main()
